# Imports and set up

In [1]:
import numpy as np
import pandas as pd

from blomap import blomap_extra_encode, add_padding

# Get the data

In [2]:
DATA_PATH = '/home/dreamtim/Coding/itmo-cpp/with_processed_sequences.csv'
# This file can be downloaded from the kaggle notebook

In [3]:
df = pd.read_csv(DATA_PATH, index_col=0)
df.head()

,sequence,extra_name,cpp_category,seq_length,is_cpp,cpp_type,origin,mol_weight,sequence_category,standard_sequence,nh3_tail,po3_pos,biotinylated,acylated_n_terminal,cyclic,amidated,stearyl_uptake,hexahistidine_tagged,modifications
0,(Acp)-KKKKKRFSFKKSFKLSGFSFKKNKK,NaN,NaN,NaN,True,NaN,NaN,NaN,Peptide with Substitutions,AKKKKKRFSFKKSFKLSGFSFKKNKK,False,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]"
3,(Acp)-RKRRQTSMTDFYHSKRRLIFS,NaN,NaN,NaN,True,NaN,NaN,NaN,Peptide with Substitutions,ARKRRQTSMTDFYHSKRRLIFS,False,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]"
7,(biotin)-lclrpvgggweaklakalakalakhlakalakalkacea,NaN,NaN,NaN,True,NaN,NaN,NaN,Biotinylated,NaN,False,False,True,False,False,False,False,False,[]
8,(biotin)-lclrpvggrsqsrsryyrqrqrsrrrrrrs,NaN,NaN,NaN,True,NaN,NaN,NaN,Biotinylated,NaN,False,False,True,False,False,False,False,False,[]
9,?A-RR-[KRRRRRE],NaN,NaN,NaN,True,NaN,NaN,NaN,Unknown,ARRKRRRRRE,False,False,False,False,False,False,False,False,[]


In [4]:
df['seq_length'] = df['sequence'].apply(lambda s: len(s) if isinstance(s, str) else 0)

# Embeddings

### Blomap

In [6]:
SEQ_LEN = df['seq_length'].max()
print(SEQ_LEN)

269


In [7]:
df['adjusted_sequence'] = df['standard_sequence'].apply(lambda s: add_padding(s, SEQ_LEN) if isinstance(s, str) else s)

In [8]:
df.head()

,sequence,extra_name,cpp_category,seq_length,is_cpp,cpp_type,origin,mol_weight,sequence_category,standard_sequence,nh3_tail,po3_pos,biotinylated,acylated_n_terminal,cyclic,amidated,stearyl_uptake,hexahistidine_tagged,modifications,adjusted_sequence
0,(Acp)-KKKKKRFSFKKSFKLSGFSFKKNKK,NaN,NaN,31,True,NaN,NaN,NaN,Peptide with Substitutions,AKKKKKRFSFKKSFKLSGFSFKKNKK,False,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]",AKKKKKRFSFKKSFKLSGFSFKKNKKXXXXXXXXXXXXXXXXXXXX...
3,(Acp)-RKRRQTSMTDFYHSKRRLIFS,NaN,NaN,27,True,NaN,NaN,NaN,Peptide with Substitutions,ARKRRQTSMTDFYHSKRRLIFS,False,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]",ARKRRQTSMTDFYHSKRRLIFSXXXXXXXXXXXXXXXXXXXXXXXX...
7,(biotin)-lclrpvgggweaklakalakalakhlakalakalkacea,NaN,NaN,48,True,NaN,NaN,NaN,Biotinylated,NaN,False,False,True,False,False,False,False,False,[],NaN
8,(biotin)-lclrpvggrsqsrsryyrqrqrsrrrrrrs,NaN,NaN,39,True,NaN,NaN,NaN,Biotinylated,NaN,False,False,True,False,False,False,False,False,[],NaN
9,?A-RR-[KRRRRRE],NaN,NaN,15,True,NaN,NaN,NaN,Unknown,ARRKRRRRRE,False,False,False,False,False,False,False,False,[],ARRKRRRRREXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...


In [9]:
# Get a non-NaN sequence
non_nan_sequence = df['adjusted_sequence'].dropna().iloc[0]

# Get the Blomap embedding for the sequence
embedding = blomap_extra_encode(non_nan_sequence)

# Get the length of the embedding
SHAPE = len(embedding)
print(SHAPE)

2959


In [10]:
blomap_embeddings = list(df['adjusted_sequence'].apply(blomap_extra_encode, args=(SHAPE,)))

/home/dreamtim/Coding/itmo-cpp/blomap.py:42: Warning: non-standard amino acid in seqeunce
  warnings.warn("non-standard amino acid in seqeunce", Warning)


In [11]:
blomap_embeddings = np.array(blomap_embeddings)
print(blomap_embeddings.shape)

(3103, 2959)


In [12]:
np.save('blomap_embeddings.npy', blomap_embeddings)

### ProtBERT

In [14]:
%%capture
%pip install transformers torch

In [17]:
import torch
from transformers import BertModel, BertTokenizer

In [18]:
# Load the pre-trained ProtBERT model and tokenizer
model_name = "Rostlab/prot_bert_bfd"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [19]:
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.current_device())  # Should return the current GPU device ID
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # GPU name

True
0
NVIDIA GeForce RTX 4050 Laptop GPU


In [20]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30, 1024, padding_idx=0)
    (position_embeddings): Embedding(40000, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-29): 30 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.0, i

In [21]:
def get_protbert_embedding(sequence):
    # Tokenize the sequence
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True, max_length=1024)

    # Move inputs to GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get embeddings from the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (the last hidden states of the model)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Average the token embeddings across the sequence
    return embeddings

In [22]:
example_sequence = df['standard_sequence'].dropna().iloc[0]
embedding = get_protbert_embedding(example_sequence)
print(embedding.shape)

torch.Size([1024])


In [26]:
# Get ProtBERT embeddings for all sequences in parallel on GPU
sequences = df['standard_sequence'].fillna('').tolist()  # Fill NaN with empty strings
batch_size = 32  # Adjust batch size based on your GPU memory

protbert_embeddings = []

for i in range(0, len(sequences), batch_size):
    batch_sequences = sequences[i:i + batch_size]
    inputs = tokenizer(batch_sequences, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move to CPU

    protbert_embeddings.append(batch_embeddings)

protbert_embeddings = np.vstack(protbert_embeddings)
print(protbert_embeddings.shape)

# Save the embeddings to a .npy file
np.save('protbert_embeddings.npy', protbert_embeddings)

(3103, 1024)


### AlphaFold

See openfold dir!

# SMILES

In [5]:
df.columns

Index(['sequence', 'extra_name', 'cpp_category', 'seq_length', 'is_cpp',
       'cpp_type', 'origin', 'mol_weight', 'sequence_category',
       'standard_sequence', 'nh3_tail', 'po3_pos', 'biotinylated',
       'acylated_n_terminal', 'cyclic', 'amidated', 'stearyl_uptake',
       'hexahistidine_tagged', 'modifications'],
      dtype='object')

In [58]:
import re
from rdkit import Chem

In [59]:
# SMILES map for common modifications
modification_smiles_map = {
    "Ac-": "CC(=O)",                # Acetyl group (N-terminal acetylation)
    "(biotin)": "C1CCCCC1NC(=O)CCC(=O)N",  # Biotin
    "-NH2": "N",                    # Amidation (C-terminal -NH2)
    "Stearyl": "CCCCCCCCCCCCCCCC(=O)",  # Stearyl group
    "Myristoyl": "CCCCCCCCCCCCCC(=O)",  # Myristoyl group
    "Lauroyl": "CCCCCCCCCCCC(=O)"       # Lauroyl group
}


In [60]:
aa_smiles_map = {
    'A': 'N[C@@H](C)C(=O)',  'a': 'N[C@H](C)C(=O)',  # Alanine
    'C': 'N[C@@H](CS)C(=O)', 'c': 'N[C@H](CS)C(=O)',  # Cysteine
    'D': 'N[C@@H](CC(=O)O)C(=O)', 'd': 'N[C@H](CC(=O)O)C(=O)',  # Aspartic acid
    'E': 'N[C@@H](CCC(=O)O)C(=O)', 'e': 'N[C@H](CCC(=O)O)C(=O)',  # Glutamic acid
    'F': 'N[C@@H](CC1=CC=CC=C1)C(=O)', 'f': 'N[C@H](CC1=CC=CC=C1)C(=O)',  # Phenylalanine
    'G': 'NCC(=O)', 'g': 'NCC(=O)',  # Glycine
    'H': 'N[C@@H](CC1=CNC=N1)C(=O)', 'h': 'N[C@H](CC1=CNC=N1)C(=O)',  # Histidine
    'I': 'N[C@@H](C(C)CC)C(=O)', 'i': 'N[C@H](C(C)CC)C(=O)',  # Isoleucine
    'K': 'N[C@@H](CCCCN)C(=O)', 'k': 'N[C@H](CCCCN)C(=O)',  # Lysine
    'L': 'N[C@@H](CC(C)C)C(=O)', 'l': 'N[C@H](CC(C)C)C(=O)',  # Leucine
    'M': 'N[C@@H](CCSC)C(=O)', 'm': 'N[C@H](CCSC)C(=O)',  # Methionine
    'N': 'N[C@@H](CC(=O)N)C(=O)', 'n': 'N[C@H](CC(=O)N)C(=O)',  # Asparagine
    'P': 'N1[C@@H](CCC1)C(=O)', 'p': 'N1[C@H](CCC1)C(=O)',  # Proline
    'Q': 'N[C@@H](CCC(=O)N)C(=O)', 'q': 'N[C@H](CCC(=O)N)C(=O)',  # Glutamine
    'R': 'N[C@@H](CCCNC(=N)N)C(=O)', 'r': 'N[C@H](CCCNC(=N)N)C(=O)',  # Arginine
    'S': 'N[C@@H](CO)C(=O)', 's': 'N[C@H](CO)C(=O)',  # Serine
    'T': 'N[C@@H](C(O)C)C(=O)', 't': 'N[C@H](C(O)C)C(=O)',  # Threonine
    'V': 'N[C@@H](C(C)C)C(=O)', 'v': 'N[C@H](C(C)C)C(=O)',  # Valine
    'W': 'N[C@@H](CC1=CNC2=CC=CC=C12)C(=O)', 'w': 'N[C@H](CC1=CNC2=CC=CC=C12)C(=O)',  # Tryptophan
    'Y': 'N[C@@H](CC1=CC=C(O)C=C1)C(=O)', 'y': 'N[C@H](CC1=CC=C(O)C=C1)C(=O)',  # Tyrosine
    'X': '*', '?': '*',  # Wildcard for unknown residues
    'O': 'N[C@@H](CCC(N)C(=O))C(=O)',  # Ornithine
}


In [61]:
# Updated modification and amino acid SMILES map
modification_smiles_map.update({
    "NII": "NC(C)C(=O)",  # Example placeholder for NII
    "PIC": "N1C=C(C(=O)N1)C",  # Example placeholder for PIC
    "IC": "NC1=NC=NC(=O)1"  # Example placeholder for IC
})

aa_smiles_map.update({
    'Aib': 'N[C@H](C(C)C)C(=O)',  # Alpha-aminoisobutyric acid
    'B': 'N[C@@H](CC(=O)O)C(=O)',  # Ambiguous (Asn/Asp as example)
    'b': 'N[C@@H](CS)C(=O)'  # Could be beta-Amino acid or placeholder
})

In [62]:
from collections import Counter

# Dictionary to count unrecognized amino acids
unrecognized_amino_acids_count = Counter()
UNRECOGNIZED_SEQUENCES = set()

# Function to handle repeats like R8 or S5
def expand_repeats(sequence):
    import re
    repeat_pattern = re.compile(r'([A-Z][a-z]*)(\d+)')
    expanded = []
    for match in repeat_pattern.finditer(sequence):
        aa, count = match.groups()
        expanded.append(aa * int(count))
    return re.sub(repeat_pattern, '', sequence) + ''.join(expanded)

# Function to convert raw sequence into SMILES
def raw_sequence_to_smiles(sequence):
    smiles_parts = []

    # Expand repeats like R8 or S5
    sequence = expand_repeats(sequence)

    # Handle modifications in the sequence
    for mod, mod_smiles in modification_smiles_map.items():
        if mod in sequence:
            sequence = sequence.replace(mod, "")
            smiles_parts.append(mod_smiles)

    # Handle the core amino acid sequence
    for aa in re.findall(r'[A-Z][a-z]*', sequence):
        if aa in aa_smiles_map:
            smiles_parts.append(aa_smiles_map[aa])
        else:
            print(f"Warning: Unrecognized amino acid or modification '{aa}' in {sequence}")
            unrecognized_amino_acids_count[aa] += 1
            UNRECOGNIZED_SEQUENCES.add(sequence)

    # Join SMILES fragments
    final_smiles = ''.join(smiles_parts)

    # Validate with RDKit
    try:
        mol = Chem.MolFromSmiles(final_smiles)
        return Chem.MolToSmiles(mol) if mol else None
    except Exception:
        return None

In [63]:
df['smiles_sequence'] = df['sequence'].apply(raw_sequence_to_smiles)

[17:13:31] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:31] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:31] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:31] Explicit valence for atom # 10 C, 5, is greater than permitted
[17:13:31] Explicit valence for atom # 10 C, 5, is greater than permitted


[17:13:31] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:31] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:31] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:32] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted
[17:13:33] Explicit valence for atom # 5 C, 5, is greater than permitted


[17:13:34] Explicit valence for atom # 5 C, 5, is greater than permitted


In [37]:
unrecognized_amino_acids_count

Counter({'O': 83,
         'Rr': 52,
         'Lys': 39,
         'Ac': 34,
         'Nspe': 34,
         'Nbtg': 20,
         'Xr': 19,
         'Et': 18,
         'His': 12,
         'Mpa': 12,
         'Cou': 12,
         'Kr': 11,
         'U': 10,
         'Grrrrrrrrr': 9,
         'Ahx': 9,
         'Npm': 8,
         'Nssb': 8,
         'Lr': 8,
         'Xk': 6,
         'Xrr': 5,
         'Cys': 5,
         'Lip': 5,
         'Qq': 5,
         'Fr': 5,
         'Kk': 5,
         'Hr': 5,
         'Bpg': 4,
         'Gr': 4,
         'Leu': 4,
         'Ct': 3,
         'Rho': 3,
         'Kf': 3,
         'Ic': 3,
         'Qa': 3,
         'Gg': 3,
         'Wl': 3,
         'Acp': 2,
         'Krrrrrrr': 2,
         'Krrrrrrrr': 2,
         'Cya': 2,
         'Rw': 2,
         'Me': 2,
         'Hey': 2,
         'Ra': 2,
         'Rh': 2,
         'Rk': 2,
         'Cl': 2,
         'Hrr': 2,
         'Ksa': 1,
         'Acetyl': 1,
         'Brr': 1,
         'Wr': 1,
    

In [64]:
for el in UNRECOGNIZED_SEQUENCES:
    print(el)

FKQqQqQqQqQq
kkwkmrrGaGrrrrrrrrr
Nbtg-Nbtg-Nspe-Nbtg-Nbtg-Nspe-Nbtg-Nbtg-Nspe-Nbtg-Nbtg-Nspe
rXrrXrrXrrXr
-GRKKRRQRRR-NHisHisHisHisHisHisHH
rRrGrKkRr
(Acp)-RKRRQTSMTDFYHSKRRLIFS
CF-GRR-cN-RR-cN-RR-cN-NAcAcAcAcAcAcHHAcAcAcAcAcAcHHAcAcAcAcAcAcHHHH
B-[KRrRrRrRrE]
eeeeeeeeeXPAANVGrrrrrrrrrXk([125I]IB)-NHH
FXrFXrFXrFXr
Mpa(luc)-LKTRVLKRWK-NHH
Mpa(luc)-KLFNKQEWTN-NHH
rRrRrRrRr
PFVYL-Lip-CouCouCouCouCouCou
CGRKKRRQRR-Ahx-RPPQ
FXrFXrFXr
AGYLLGK[Ksa4qn4]INLKALAALAKKILKK
FXrFXrFXrFXrFXr
Me-RKKRRRESWVHLPPPVHLPPPGGHHHHHH
stearly--HHHHHHHHHHHHHHHH–NHisHisHisHisHisHisHH
KrRrRrRrRrRELTFKEYWDQLTSAA
rFTFHFrFEFTFHFE
CGRKKRRQR-Ahx-RRPPQ
CRWRwKCCKK
NLys-NLys-NLys-NLys-NLys-NLys-NLys-NLys-NLeu-NLeu-NLeu-NLeu-Nleu
FFFFFFGGGGGGrrrRR
?A-[KrrrrrrrrE]
BrrXrrXrrX
GPLGIAGQrrrrrrrrr
KrRrGgKkRrDLTFKEYWDQLTSAA
rrrRrRrR
RRRRRRRR-H-NPs-siScr
CF-GRR-cN-RR-cN-RR-cN-NAcAcAcAcAcHHAcAcAcAcAcHHAcAcAcAcAcHHHH
Cys-RRRRRRRR-dGR-Lip
Acetyl-RLWRLLWRLWRRLWRLLR-NHH
RaDPAYQaRFLGKrKfIcL
Tf-PFVYLI
LAELLAELLAELGGGGrrrrrrrrr
eeeeeeeeeX

In [65]:
# Update modification and amino acid SMILES map to handle the unrecognized amino acids
modification_smiles_map.update({
    "Nspe": "N[C@H](C(C)O)C(=O)",  # Placeholder for N-spe (needs actual structure)
    "Nbtg": "N[C@H](C(C)(C)C)C(=O)",  # Placeholder for N-btg (needs actual structure)
    "Ac": "CC(=O)",  # Acetyl group (already in the original map, handling here for completeness)
    "Et": "CC",  # Ethyl group (modification, placeholder)
    "Npm": "N1[C@H](C(C)C)C(=O)",  # Placeholder for Npm (needs actual structure)
    "Nssb": "N1[C@H](C(C)CC)C(=O)",  # Placeholder for Nssb (needs actual structure)
    "Mpa": "NC(=O)C(C)C(C)C(=O)",  # Placeholder for Mpa (needs actual structure)
    "Cou": "C1=CC2=C(C=C1)C(=O)O2",  # Placeholder for Cou (needs actual structure)
    "Xr": "N[C@H](C(C)C)C(=O)",  # Placeholder for Xr (needs actual structure)
    "His": "N[C@@H](CC1=CNC=N1)C(=O)",  # Histidine (already handled in the original map, handling here for completeness)
})

aa_smiles_map.update({
    'Rr': 'N[C@@H](CC(C)C)C(=O)',  # Placeholder for Rr (represents an amino acid sequence with R and R-like groups)
    'Lys': 'N[C@H](CCCCN)C(=O)',  # Lysine (already in original map)
    'Grrrrrrrrr': 'N[C@@H](C(C)CCCCCCCCCC)C(=O)',  # Placeholder for Grrrrrrrrr, assume a modified version
    'Ahx': 'N[C@H](CC(C)CC)C(=O)',  # Placeholder for Ahx (needs actual structure)
    'Kr': 'N[C@H](CC(C)CN)C(=O)',  # Placeholder for Kr (needs actual structure)
    'Lr': 'N[C@H](CC(C)C(C)C)C(=O)',  # Placeholder for Lr (needs actual structure)
    'U': 'NC1=NC2=C(C=C1)C(=O)N2',  # Placeholder for U (assumed modified structure)
})


In [100]:
# Dictionary to count unrecognized amino acids
unrecognized_amino_acids_count = Counter()
UNRECOGNIZED_SEQUENCES = set()

# Function to handle repeats like R8 or S5
def expand_repeats(sequence):
    repeat_pattern = re.compile(r'([A-Z][a-z]*)(\d+)')
    expanded = []
    for match in repeat_pattern.finditer(sequence):
        aa, count = match.groups()
        expanded.append(aa * int(count))
    return re.sub(repeat_pattern, '', sequence) + ''.join(expanded)

# Function to convert raw sequence into SMILES
def raw_sequence_to_smiles(sequence):
    smiles_parts = []

    # Expand repeats like R8 or S5
    sequence = expand_repeats(sequence)

    # Handle modifications in the sequence
    for mod, mod_smiles in modification_smiles_map.items():
        if mod in sequence:
            sequence = sequence.replace(mod, "")
            smiles_parts.append(mod_smiles)

    # Handle the core amino acid sequence
    for aa in re.findall(r'[A-Z][a-z]*', sequence):
        if aa in aa_smiles_map:
            smiles_parts.append(aa_smiles_map[aa])
        else:
            unrecognized_amino_acids_count[aa] += 1
            UNRECOGNIZED_SEQUENCES.add(sequence)

    # Join SMILES fragments
    final_smiles = ''.join(smiles_parts)

    # Validate with RDKit
    try:
        mol = Chem.MolFromSmiles(final_smiles)
        return Chem.MolToSmiles(mol) if mol else None
    except Exception:
        return None

Final function

In [101]:
def sequence_to_smiles(row):
    try:
        mol = Chem.MolFromSequence(row['sequence'])
        if not mol:
            return raw_sequence_to_smiles(row['sequence'])
        if mol:
            return Chem.MolToSmiles(mol) if mol else None
    except Exception as e:
        return raw_sequence_to_smiles(row['sequence'])


In [102]:
df['smiles_sequence'] = df.apply(sequence_to_smiles, axis=1)

[17:25:55] Explicit valence for atom # 10 C, 5, is greater than permitted
[17:25:55] Explicit valence for atom # 10 C, 5, is greater than permitted
[17:25:56] SMILES Parse Error: unclosed ring for input: 'N1[C@H](C(C)CC)C(=O)N[C@@H](CC(=O)N)C(=O)N[C@H](CCCCN)C(=O)N[C@@H](CC(=O)N)C(=O)N[C@H](CCCCN)C(=O)N[C@@H](CC(=O)N)C(=O)N[C@H](CCCCN)C(=O)N[C@@H](CC(=O)N)C(=O)N[C@H](CCCCN)C(=O)'
[17:25:56] Explicit valence for atom # 8 O, 3, is greater than permitted
[17:25:56] Explicit valence for atom # 8 O, 3, is greater than permitted


In [103]:
df.head()

,sequence,extra_name,cpp_category,seq_length,is_cpp,cpp_type,origin,mol_weight,sequence_category,standard_sequence,...,po3_pos,biotinylated,acylated_n_terminal,cyclic,amidated,stearyl_uptake,hexahistidine_tagged,modifications,smiles_sequence,is_valid_smiles
0,(Acp)-KKKKKRFSFKKSFKLSGFSFKKNKK,NaN,NaN,31,True,NaN,NaN,NaN,Peptide with Substitutions,AKKKKKRFSFKKSFKLSGFSFKKNKK,...,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]",CC(=O)N[C@@H](CCCCN)C(=O)N[C@@H](CCCCN)C(=O)N[...,True
3,(Acp)-RKRRQTSMTDFYHSKRRLIFS,NaN,NaN,27,True,NaN,NaN,NaN,Peptide with Substitutions,ARKRRQTSMTDFYHSKRRLIFS,...,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]",CCC(C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCC...,True
7,(biotin)-lclrpvgggweaklakalakalakhlakalakalkacea,NaN,NaN,48,True,NaN,NaN,NaN,Biotinylated,NaN,...,False,True,False,False,False,False,False,[],NC(=O)CCC(=O)NC1CCCCC1,True
8,(biotin)-lclrpvggrsqsrsryyrqrqrsrrrrrrs,NaN,NaN,39,True,NaN,NaN,NaN,Biotinylated,NaN,...,False,True,False,False,False,False,False,[],NC(=O)CCC(=O)NC1CCCCC1,True
9,?A-RR-[KRRRRRE],NaN,NaN,15,True,NaN,NaN,NaN,Unknown,ARRKRRRRRE,...,False,False,False,False,False,False,False,[],C[C@H](N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](...,True


In [104]:
# Check if SMILES are not NaN
valid_smiles = df['smiles_sequence'].notna()

# Calculate the percentage of valid SMILES
valid_percentage = valid_smiles.mean() * 100
invalid_percentage = 100 - valid_percentage

# Print the percentages
print(f'Percentage of Valid SMILES: {valid_percentage:.2f}%')
print(f'Percentage of Invalid SMILES: {invalid_percentage:.2f}%')


Percentage of Valid SMILES: 99.84%
Percentage of Invalid SMILES: 0.16%


In [111]:
from rdkit import Chem

def is_valid_smiles(smiles):
    if not smiles:
        return False
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

# Apply the validation function to the 'smiles_sequence' column
df['is_valid_smiles'] = df['smiles_sequence'].apply(lambda x: is_valid_smiles(x) if pd.notna(x) else False)

# Calculate the percentage of valid SMILES
valid_percentage = df['is_valid_smiles'].mean() * 100

# Print the percentage of valid SMILES
print(f'Percentage of Valid SMILES: {valid_percentage:.2f}%')

# Check if all SMILES sequences are valid
if valid_percentage == 100:
    print("ALL SMILES ARE VALID!")

# Print the DataFrame with the new column
df.head()

Percentage of Valid SMILES: 99.84%


,sequence,extra_name,cpp_category,seq_length,is_cpp,cpp_type,origin,mol_weight,sequence_category,standard_sequence,...,po3_pos,biotinylated,acylated_n_terminal,cyclic,amidated,stearyl_uptake,hexahistidine_tagged,modifications,smiles_sequence,is_valid_smiles
0,(Acp)-KKKKKRFSFKKSFKLSGFSFKKNKK,NaN,NaN,31,True,NaN,NaN,NaN,Peptide with Substitutions,AKKKKKRFSFKKSFKLSGFSFKKNKK,...,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]",CC(=O)N[C@@H](CCCCN)C(=O)N[C@@H](CCCCN)C(=O)N[...,True
3,(Acp)-RKRRQTSMTDFYHSKRRLIFS,NaN,NaN,27,True,NaN,NaN,NaN,Peptide with Substitutions,ARKRRQTSMTDFYHSKRRLIFS,...,False,False,False,False,False,False,False,"[('C', 2), ('P', 3)]",CCC(C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCC...,True
7,(biotin)-lclrpvgggweaklakalakalakhlakalakalkacea,NaN,NaN,48,True,NaN,NaN,NaN,Biotinylated,NaN,...,False,True,False,False,False,False,False,[],NC(=O)CCC(=O)NC1CCCCC1,True
8,(biotin)-lclrpvggrsqsrsryyrqrqrsrrrrrrs,NaN,NaN,39,True,NaN,NaN,NaN,Biotinylated,NaN,...,False,True,False,False,False,False,False,[],NC(=O)CCC(=O)NC1CCCCC1,True
9,?A-RR-[KRRRRRE],NaN,NaN,15,True,NaN,NaN,NaN,Unknown,ARRKRRRRRE,...,False,False,False,False,False,False,False,[],C[C@H](N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](...,True


In [109]:
BAD_SEQUENCES = """
MGVADLIKKFESISKEEGGGGK(RhB)GGrRrRrRRR
A-C-Bpg-LYLLGKINLKALAALAKKILFFF
eeeeeeeeeXPAACtVGrrrrrrrrrXk(FL)-NHH
H-RRRRRRK-Rho-NHH
eeeeeeeeeXPAANVGrrrrrrrrrXk(FL)-NHH
M-Rho-GRKKRRQRRR-NHH
Z-LRGG-AMC
B-[KrrrrrrrE]
RKKRRQRRR-OL-Pdna
AGY-C-Bpg-LGKINLKALAALAKKILFFF
PFVYL-Lip-
agy-C-Bpg-lgkinlkalaalakkilFFF
GD(Abu)LPHLKLC
FSDMeSSSVPNBBRNCG
BrrrrX
eeeeeeeeeXPAANVGrrrrrrrrrXk([125I]IB)-NHH
?A-[KrrrrrrrE]
ε-(Steary-AGYLLG)KINLKALAALAKKIL
XPAACtVGrrrrrrrrrXk(FL)-NHH
CF-Cys-BLGTYTQDFNXFHTFPQTAIGVGAP
AGYLLGK[Ksa4qn4]INLKALAALAKKILKK
Cyclo(FΦRRRRQ)
eeeeeeeeeXPAACtVGrrrrrrrrrXk([125I]IB)-NHH
MGVADLIKKFESISKEEGGGGK(Ahx-FITC)-GGrRrRrRRR
B-[KrrrrrrrrE]
MGVADLIKKFESISKEEGGGGK(PA-RhB)GGrRrRrRRR
NLys-NLys-NLys-NLys-NLys-NLys-NLys-NLys-NLeu-NLeu-NLeu-NLeu-Nleu
XPAANVGrrrrrrrrrXk(FL)-NHH
VGAlAvVvWlWlWlWbA-GSG-PKKKRKVC
?A-[KrrrrrrrrE]
RRRRRRRR-c(RGDfK)
Dabcyl-RRRRRRK-Rho-NHH
a-C-Bpg-lyllgkinlkalaalakkilFFF
"""

In [114]:
df.to_csv('with_smiles.csv', index=False)

# SMILES based embeddings

In [128]:
from rdkit.Chem import AllChem

counter = 0

def get_morgan_fingerprint(smiles, radius=2, nBits=1024):
    """Generate a Morgan fingerprint from a SMILES string."""
    try:
        molecule = Chem.MolFromSmiles(smiles)
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=nBits)
        return np.array(fingerprint)
    except Exception:
        global counter
        counter += 1
        return np.zeros(nBits)  # Return a zero array of the desired shape if the SMILES is invalid

# Generate fingerprints for all SMILES
fingerprints = []

for smiles in df['smiles_sequence']:
    fingerprint = get_morgan_fingerprint(smiles)
    fingerprints.append(fingerprint)

[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerator
[21:11:22] DEPRECATION WARNING: please use MorganGenerat

In [129]:
# Convert list of fingerprints to a NumPy array
fingerprints_array = np.array(fingerprints)

print(fingerprints_array.shape)  # Print the shape of the resulting array
print('Empty num:', counter)

# Save the fingerprints to a .npy file
np.save('morgan_fingerprints.npy', fingerprints_array)

(3103, 1024)
Empty num: 5
